# Parallel workflows in programs

In this document, we will learn how to run distributed workflows inside a program. In this case, we will compute the quasi-probability distribution in parallel for a list of quantum circuits.

Let's take a look at the program file [./source_files/program_2.py](./source_files/program_2.py). 

```python
# source_files/program_2.py

from quantum_serverless import get_arguments, save_result, distribute_task, get

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler


@distribute_task()
def distributed_sample(circuit: QuantumCircuit):
    """Distributed task that returns quasi distribution for given circuit."""
    return Sampler().run(circuit).result().quasi_dists[0]


arguments = get_arguments()
circuits = arguments.get("circuits")


# run distributed tasks as async function
# we get task references as a return type
sample_task_references = [
    distributed_sample(circuit)
    for circuit in circuits
]

# now we need to collect results from task references
results = get(sample_task_references)

save_result({
    "results": results
})
```

There are a lot of new concepts introduced in this program, so let's go over them in more detail:

The `distribute_task` decorator converts a function into a distributed task. This means that the function will be executed on compute resources asynchronously and in parallel to the main context of the program.

When you call a converted function, it will return a reference to the function execution instead of the result. In order to get the result back, you need to call the `get` function on the function reference. `get` will wait until the function is finished and return the result of the function execution.

In the program above, we have applied the `distribute_task` decorator to the `distributed_sample` function. This function takes a `QuantumCircuit` as input and returns the quasi distribution for that circuit.

After we have defined the `distributed_sample` function, we read the circuits from the program arguments using the `get_arguments` function. We then call the `distributed_sample` function for each of the circuits, which creates a reference to each of the function executions.

These function executions will run in parallel on compute resources, and we get task references as the return type. We store these task references in the `sample_task_references` list.

After we have created the task references for each of the function executions, we need to collect the results from these tasks. We do this by calling the `get` function on the list of task references, which waits until all the tasks have completed and returns the results.

Once we have the results, we can save them using the `save_result` function.

Essentially, this program reads the circuits from the program arguments, executes the `distributed_sample` function on each circuit in parallel, collects the results from the function executions, and saves the results.

In [ ]:
from quantum_serverless import QuantumServerless, Provider
import os

In [ ]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

Let's create list of random circuit which we will be using as arguments

In [ ]:
from qiskit.circuit.random import random_circuit

circuits = [random_circuit(2, 2) for _ in range(3)]
[circuit.measure_all() for circuit in circuits]
circuits

Run program as usual

In [ ]:
from quantum_serverless import Program

program = Program(
    title="Program with distributed workflow",
    entrypoint="program_2.py",
    working_dir="./source_files/"
)

job = serverless.run(program, arguments={"circuits": circuits})
job

In [ ]:
job.status()

In [ ]:
job.result()